In [3]:
import glob
import os
from pydub import AudioSegment

In [4]:
import pandas as pd
import librosa
import librosa.display

In [5]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        flux = get_spectral_features(audio, sample_rate)[2]
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         spectral_features = get_spectral_features(audio, sample_rate)
        chroma_features = get_chroma_features(audio, sample_rate)
        return [mfccs, flux] + chroma_features
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None

In [6]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'flux', 'chroma_features1', 'chroma_features2', 'chroma_features3', 'class_label'])
    return data

In [66]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [67]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [68]:
def get_spectral_features(audio, sr):
    sp = librosa.feature.spectral_centroid(audio)
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    sp_scaled = np.mean(sp.T,axis=0)
    a_scaled = np.mean(a.T,axis=0)
    onset_env_scaled = np.mean(onset_env,axis=0)
    return [sp_scaled, a_scaled, onset_env_scaled]

In [69]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [70]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [71]:
data = get_all_features('data')

In [142]:
data = data.sample(frac=1).reset_index(drop=True)
data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-611.8136, 40.777424, 12.620239, 20.898668, 1...",[1.0362669],"[0.4842381, 0.44837013, 0.41868302, 0.38146535...","[0.2662793381049647, 0.25075628730815097, 0.22...","[0.5629649, 0.49578902, 0.47771728, 0.42898136...",4
1,"[-483.8801, 67.22944, -3.035094, 23.410534, 10...",[1.2832025],"[0.5137478, 0.35476452, 0.25112936, 0.2839159,...","[0.39973669152152785, 0.34634047732841317, 0.1...","[0.77134436, 0.6558588, 0.44797945, 0.49658856...",5
2,"[-529.7939, 111.131096, 24.161963, 41.378025, ...",[0.97019327],"[0.26051733, 0.30142537, 0.2903194, 0.2532308,...","[0.135444049117274, 0.17529732278287938, 0.180...","[0.32595995, 0.3542648, 0.34927756, 0.32550025...",2
3,"[-524.6342, 69.502625, 10.729943, 26.06364, 7....",[1.5747317],"[0.64704764, 0.6812321, 0.6889332, 0.64720935,...","[0.3859249430913798, 0.44412299170113206, 0.35...","[0.7365093, 0.85648644, 0.69298464, 0.5742491,...",4
4,"[-551.78143, 107.96993, 17.979797, 14.39397, 1...",[1.0554308],"[0.46269706, 0.52385974, 0.5351941, 0.59048325...","[0.3333913937522035, 0.3589864045660255, 0.313...","[0.70506465, 0.73798627, 0.66210145, 0.6403810...",5
...,...,...,...,...,...,...
933,"[-509.2625, 96.30729, 13.490375, 19.20398, 6.8...",[1.2931988],"[0.37592518, 0.34144098, 0.377211, 0.39135712,...","[0.28603218673667047, 0.23617766442427635, 0.2...","[0.592766, 0.49719438, 0.43252078, 0.42320997,...",1
934,"[-525.10944, 98.93431, -8.021024, 20.28867, -8...",[1.5247883],"[0.49913427, 0.28826502, 0.20493266, 0.1747037...","[0.47062531551012093, 0.339033196969182, 0.199...","[0.76767975, 0.5960087, 0.4140232, 0.36414152,...",2
935,"[-374.46118, 87.18423, 14.456073, 25.931875, 1...",[1.0921358],"[0.6858864, 0.7387477, 0.6847408, 0.65038735, ...","[0.39391395897289844, 0.4267180250345143, 0.37...","[0.8094848, 0.86993295, 0.75949717, 0.66599727...",5
936,"[-549.74994, 80.772, 37.733208, 32.31267, 18.5...",[0.85393745],"[0.60667306, 0.5226306, 0.50657195, 0.5327808,...","[0.3626017982974113, 0.2484315618862962, 0.215...","[0.7378085, 0.5837045, 0.54220885, 0.58441585,...",2


In [143]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
# testLabels = [[i] for i in testData['class_label']]
# y_train = np.array(labels)
# y_test = np.array(testLabels)

x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.1, random_state = 127)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(844, 6) (94, 6) (844, 1) (94, 1)


# One-hot encode

In [144]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [145]:
y_onehot_train = get_one_hot(y_train)
y_onehot_test = get_one_hot(y_test)
print(y_onehot_train.shape, y_onehot_test.shape)

(844, 5) (94, 5)


/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "ca

# PCA

In [146]:
def pca(X):
    # normalize the features
    X = (X - X.mean()) / X.std()
    
    # compute the covariance matrix
    X = np.matrix(X)
    cov = (X.T * X) / X.shape[0]
    
    # perform SVD
    U, S, V = np.linalg.svd(cov)
    
    return U, S, V

In [147]:
def project_data(X, U, k):
    U_reduced = U[:,:k]
    return np.dot(X, U_reduced)

In [148]:
# def recover_data(Z, U, k):
#     U_reduced = U[:,:k]
#     return np.dot(Z, U_reduced.T)

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [149]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
    return res

In [150]:
ls_train = get_processed_X(x_train)
ls_test = get_processed_X(x_test)
print(ls_train.shape, ls_test.shape)

(844, 137) (94, 137)


In [151]:
k = 500
U, S, V = pca(ls_train)
ls_train = project_data(ls_train, U, k)
U1, S1, V1 = pca(ls_test)
ls_test = project_data(ls_test, U, k)

 # ნეირონული ქსელის კოდი

In [152]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [153]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [154]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = []
    for i in range(z3.shape[0]):
        z = [z3[i, j] for j in range(z3.shape[1])]
        h.append(softmax(z))
    h = np.array(h)
#     h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [155]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

#     J = (-1 / m) * np.sum(y * np.log(h).T) #softmax
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [156]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [157]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = cost(params,input_size, hidden_size, num_labels, X, y, learning_rate)

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [158]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 30
num_labels = y_onehot_train.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((30, 138), (5, 31))

In [159]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape
print(h)
print(cost(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate))

[[0.12318941 0.20553901 0.23432169 0.15344757 0.28350231]
 [0.12620657 0.22003498 0.21971832 0.13558142 0.29845871]
 [0.12940829 0.21962338 0.22240396 0.13442971 0.29413467]
 ...
 [0.1248937  0.20143688 0.2367653  0.14779189 0.28911223]
 [0.12619905 0.20997429 0.22846577 0.14048264 0.29487824]
 [0.12135105 0.20048803 0.23579834 0.14608607 0.29627651]]
2.5763986528808616


In [160]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(2.5763986528808616, (4295,))

In [161]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 500})
fmin

/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


     fun: 1.3895781748780511
     jac: array([ 2.99945401e-10, -6.67141655e-05,  4.16573090e-05, ...,
       -1.79069667e-04, -1.67753319e-04, -4.63298809e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 500
     nit: 61
  status: 3
 success: False
       x: array([ 0.12204956, -0.0563647 ,  0.03516492, ..., -0.1511348 ,
       -0.1415838 , -0.03910242])

In [117]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
y_pred_train = np.array(np.argmax(h, axis=1) + 1)

In [118]:
ls_test = np.matrix(ls_test)

a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

In [119]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.7485970819304153


In [120]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.5106382978723404


# ჩაწერა-წაკითხვა

In [36]:
# thetas = []
# thetas.append(theta1)
# thetas.append(theta2)
# np.save('weights', thetas)
# theta1, theta2 = np.load('weights.npy', allow_pickle = True)